In [4]:
import pandas as pd

file_path = '/content/drive/MyDrive/data retail sale/Online Retail.xlsx'
df = pd.read_excel(file_path)

# Xem 5 dòng đầu
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
# 1. Xoá các dòng bị thiếu thông tin quan trọng
df = df.dropna(subset=['CustomerID', 'Description'])

# 2. Chuẩn hoá văn bản trong cột Description
df['Description'] = df['Description'].str.strip().str.upper()

# 3. Loại bỏ các giao dịch có Quantity <= 0 hoặc UnitPrice <= 0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 4. Tạo cột TotalPrice = Quantity * UnitPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


<ipython-input-9-4f00533e0671>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [10]:
# Chuyển cột InvoiceDate sang kiểu datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Tạo thêm các cột liên quan đến thời gian
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour


In [11]:
df['CustomerID'] = df['CustomerID'].astype(int)


In [24]:
# Thống kê dữ liệu sau xử lý
print(" Dữ liệu sau làm sạch:")
print(df.info())

# Xem trước 5 dòng đầu
df.head()
df.to_csv('/content/drive/MyDrive/data retail sale/Online Retail Cleaned.csv', index=False)


 Dữ liệu sau làm sạch:
<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  int64         
 7   Country      397884 non-null  object        
 8   TotalPrice   397884 non-null  float64       
 9   Year         397884 non-null  int32         
 10  Month        397884 non-null  int32         
 11  Day          397884 non-null  int32         
 12  Hour         397884 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(4)
memory usage: 36.4+ MB
None


In [19]:
# PHÂN TÍCH 2: GÓC NHÌN SẢN PHẨM (PRODUCT ANALYSIS)


# 1. Top 10 sản phẩm bán chạy nhất theo số lượng
top_products_by_qty = (
    df.groupby('Description')['Quantity']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'Quantity': 'Tổng số lượng bán'})
)

# 2. Top 10 sản phẩm có doanh thu cao nhất
top_products_by_revenue = (
    df.groupby('Description')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'TotalPrice': 'Tổng doanh thu'})
)

# 3. Giá bán trung bình theo sản phẩm (lọc các sản phẩm bán từ 100 lần trở lên để loại nhiễu)
avg_price = (
    df.groupby('Description')
    .agg(Số_lần_bán=('Quantity', 'count'), Giá_trung_bình=('UnitPrice', 'mean'))
    .query('Số_lần_bán >= 100')
    .sort_values(by='Giá_trung_bình', ascending=False)
    .reset_index()
)

# Hiển thị top 10 sản phẩm bán chạy nhất
print("🔹 Top 10 sản phẩm bán chạy nhất theo số lượng:")
display(top_products_by_qty)

# Hiển thị top 10 sản phẩm doanh thu cao nhất
print("\n🔹 Top 10 sản phẩm có doanh thu cao nhất:")
display(top_products_by_revenue)

# Hiển thị giá bán trung bình
print("\n🔹 Giá bán trung bình của sản phẩm (lọc từ 100 lượt trở lên):")
display(avg_price.head(10))  # hiển thị 10 dòng đầu


🔹 Top 10 sản phẩm bán chạy nhất theo số lượng:


,Description,Tổng số lượng bán
0,"PAPER CRAFT , LITTLE BIRDIE",80995
1,MEDIUM CERAMIC TOP STORAGE JAR,77916
2,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54415
3,JUMBO BAG RED RETROSPOT,46181
4,WHITE HANGING HEART T-LIGHT HOLDER,36725
5,ASSORTED COLOUR BIRD ORNAMENT,35362
6,PACK OF 72 RETROSPOT CAKE CASES,33693
7,POPCORN HOLDER,30931
8,RABBIT NIGHT LIGHT,27202
9,MINI PAINT SET VINTAGE,26076



🔹 Top 10 sản phẩm có doanh thu cao nhất:


,Description,Tổng doanh thu
0,"PAPER CRAFT , LITTLE BIRDIE",168469.60
1,REGENCY CAKESTAND 3 TIER,142592.95
2,WHITE HANGING HEART T-LIGHT HOLDER,100448.15
3,JUMBO BAG RED RETROSPOT,85220.78
4,MEDIUM CERAMIC TOP STORAGE JAR,81416.73
5,POSTAGE,77803.96
6,PARTY BUNTING,68844.33
7,ASSORTED COLOUR BIRD ORNAMENT,56580.34
8,MANUAL,53779.93
9,RABBIT NIGHT LIGHT,51346.20



🔹 Giá bán trung bình của sản phẩm (lọc từ 100 lượt trở lên):


,Description,Số_lần_bán,Giá_trung_bình
0,MANUAL,284,175.291585
1,CARRIAGE,133,50.135338
2,SET/4 WHITE RETRO STORAGE CUBES,106,38.865094
3,POSTAGE,1099,31.570482
4,BREAD BIN DINER STYLE IVORY,341,16.527947
5,SPACEBOY BABY GIFT SET,143,16.516434
6,DOLLY GIRL BABY GIFT SET,122,16.425410
7,BREAD BIN DINER STYLE RED,249,16.345382
8,BREAD BIN DINER STYLE MINT,120,16.272333
9,BREAD BIN DINER STYLE PINK,112,16.215446


In [20]:
# PHÂN TÍCH 3: GÓC NHÌN KHÁCH HÀNG (CUSTOMER ANALYSIS)

# 1. Top 10 khách hàng chi tiêu nhiều nhất
top_customers = (
    df.groupby('CustomerID')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'TotalPrice': 'Tổng chi tiêu'})
)

# 2. Tần suất mua hàng của từng khách hàng
purchase_frequency = (
    df.groupby('CustomerID')['InvoiceNo']
    .nunique()
    .reset_index()
    .rename(columns={'InvoiceNo': 'Số lần mua hàng'})
    .sort_values(by='Số lần mua hàng', ascending=False)
)

# 3. Doanh thu theo quốc gia
revenue_by_country = (
    df.groupby('Country')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'TotalPrice': 'Tổng doanh thu'})
)

# 4. Số lượng khách hàng theo quốc gia
customers_by_country = (
    df.groupby('Country')['CustomerID']
    .nunique()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'CustomerID': 'Số khách hàng'})
)

# Hiển thị kết quả
print("🔹 Top 10 khách hàng chi tiêu nhiều nhất:")
display(top_customers)

print("\n🔹 Tần suất mua hàng của khách hàng (Top 10):")
display(purchase_frequency.head(10))

print("\n🔹 Doanh thu theo quốc gia (Top 10):")
display(revenue_by_country.head(10))

print("\n🔹 Số lượng khách hàng theo quốc gia (Top 10):")
display(customers_by_country.head(10))


🔹 Top 10 khách hàng chi tiêu nhiều nhất:


,CustomerID,Tổng chi tiêu
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
5,12415,124914.53
6,14156,117379.63
7,17511,91062.38
8,16029,81024.84
9,12346,77183.60



🔹 Tần suất mua hàng của khách hàng (Top 10):


,CustomerID,Số lần mua hàng
326,12748,209
1879,14911,201
4010,17841,124
562,13089,97
1661,14606,93
2176,15311,91
481,12971,86
1689,14646,73
2702,16029,63
795,13408,62



🔹 Doanh thu theo quốc gia (Top 10):


,Country,Tổng doanh thu
0,United Kingdom,7308391.554
1,Netherlands,285446.340
2,EIRE,265545.900
3,Germany,228867.140
4,France,209024.050
5,Australia,138521.310
6,Spain,61577.110
7,Switzerland,56443.950
8,Belgium,41196.340
9,Sweden,38378.330



🔹 Số lượng khách hàng theo quốc gia (Top 10):


,Country,Số khách hàng
0,United Kingdom,3920
1,Germany,94
2,France,87
3,Spain,30
4,Belgium,25
5,Switzerland,21
6,Portugal,19
7,Italy,14
8,Finland,12
9,Austria,11


In [21]:
# PHÂN TÍCH 4: GÓC NHÌN THEO THỜI GIAN (TIME ANALYSIS)

# 1. Doanh thu theo giờ trong ngày
revenue_by_hour = (
    df.groupby('Hour')['TotalPrice']
    .sum()
    .reset_index()
    .sort_values(by='Hour')
    .rename(columns={'TotalPrice': 'Tổng doanh thu'})
)

# 2. Doanh thu theo ngày trong tháng
revenue_by_day = (
    df.groupby('Day')['TotalPrice']
    .sum()
    .reset_index()
    .sort_values(by='Day')
    .rename(columns={'TotalPrice': 'Tổng doanh thu'})
)

# 3. Tổng doanh thu theo tháng
revenue_by_month_only = (
    df.groupby('Month')['TotalPrice']
    .sum()
    .reset_index()
    .sort_values(by='Month')
    .rename(columns={'TotalPrice': 'Tổng doanh thu'})
)

# Hiển thị kết quả
print("🔹 Doanh thu theo giờ trong ngày:")
display(revenue_by_hour)

print("\n🔹 Doanh thu theo ngày trong tháng:")
display(revenue_by_day)

print("\n🔹 Doanh thu theo tháng trong năm:")
display(revenue_by_month_only)


🔹 Doanh thu theo giờ trong ngày:


,Hour,Tổng doanh thu
0,6,4.250
1,7,31059.210
2,8,282115.630
3,9,842605.171
4,10,1261192.571
5,11,1104558.750
6,12,1378571.480
7,13,1173264.750
8,14,995629.371
9,15,966191.750



🔹 Doanh thu theo ngày trong tháng:


,Day,Tổng doanh thu
0,1,286370.870
1,2,256858.750
2,3,285430.150
3,4,329347.220
4,5,354033.810
5,6,336759.840
6,7,376788.980
7,8,296747.450
8,9,460216.080
9,10,325106.180



🔹 Doanh thu theo tháng trong năm:


,Month,Tổng doanh thu
0,1,569445.040
1,2,447137.350
2,3,595500.760
3,4,469200.361
4,5,678594.560
5,6,661213.690
6,7,600091.011
7,8,645343.900
8,9,952838.382
9,10,1039318.790


In [23]:
# PHÂN TÍCH 5: HÀNH VI TRẢ HÀNG (RETURN / REFUND ANALYSIS)

# Các giao dịch trả hàng có InvoiceNo bắt đầu bằng 'C' (Credit Note)
returns = df[df['InvoiceNo'].astype(str).str.startswith('C')]

# 1. Tổng số đơn hàng bị trả
total_return_orders = returns['InvoiceNo'].nunique()

# 2. Tổng giá trị bị trả
total_return_value = returns['TotalPrice'].sum()

# 3. Tỷ lệ hoàn hàng trên toàn bộ đơn hàng
total_orders = df['InvoiceNo'].nunique()
return_rate = round((total_return_orders / total_orders) * 100, 2)

# 4. Top 10 sản phẩm bị trả nhiều nhất
top_returned_products = (
    returns.groupby('Description')['Quantity']
    .sum()
    .abs()  # Giá trị Quantity trong đơn trả là âm
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'Quantity': 'Tổng số lượng bị trả'})
)

# 5. Quốc gia có số hoàn hàng cao nhất
top_returned_countries = (
    returns.groupby('Country')['InvoiceNo']
    .nunique()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'InvoiceNo': 'Số đơn bị trả'})
)

# Hiển thị kết quả
print(" Tổng số đơn hàng bị trả:", total_return_orders)
print(" Tổng giá trị bị trả: £", round(total_return_value, 2))
print(" Tỷ lệ hoàn hàng:", return_rate, "%")

print("\n Top 10 sản phẩm bị trả nhiều nhất:")
display(top_returned_products)

print("\n Quốc gia có số đơn bị trả nhiều nhất:")
display(top_returned_countries)


 Tổng số đơn hàng bị trả: 0
 Tổng giá trị bị trả: £ 0.0
 Tỷ lệ hoàn hàng: 0.0 %

 Top 10 sản phẩm bị trả nhiều nhất:


,Description,Tổng số lượng bị trả



 Quốc gia có số đơn bị trả nhiều nhất:


,Country,Số đơn bị trả
